In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import torch
import json
from tqdm import tqdm
from evaluate import load
import time

2024-11-22 08:27:51.087878: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 08:27:51.281656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 08:27:51.355903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 08:27:51.376839: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 08:27:51.517083: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
# Step 1: Load the tokenizer and model with quantization
dataset = load_dataset("google/IFEval")
# model_path = "Qwen/Qwen2.5-0.5B-Instruct"  # Near 3B model (smallest available Qwen model)
model_path = '../models/best_model/checkpoint-100'
# model_path = 'v2_mistral7b_results/checkpoint-100'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

compute_dtype = getattr(torch, "float32")  # Set computation data type to bfloat16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Enable loading the model in 4-bit precision
    bnb_4bit_quant_type="nf4",            # Specify quantization type as Normal Float 4
    bnb_4bit_compute_dtype=compute_dtype, # Set computation data type
    bnb_4bit_use_double_quant=True,       # Use double quantization for better accuracy
)



model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map=device,
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def tag_message(message, role="prompter"):
    return f"<{role}> {message} </{role}>"

In [4]:
# Step 3: Generate predictions on the dataset
output_file = "mistral_final.jsonl"
model.eval()
with open(output_file, 'w', encoding='utf-8') as f_out:
    for sample in tqdm(dataset['train'].select(range(20))):   # Use 'validation' or 'train' split if 'test' is not available
        input_text = sample['prompt']  # Adjust the field name based on the dataset's structure

        # Prepare the input prompt
        prompt = input_text

        # Tokenize input
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        # Generate output
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=512,
            eos_token_id=tokenizer.eos_token_id,
        )

        # Decode output
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Since the model may include the prompt in its output, we extract the generated response
        response = generated_text[len(prompt):]

        # Prepare the JSON object
        json_obj = {
            "prompt": prompt,
            "response": response
        }
        print(json_obj)
        # Write the JSON object to file
        # f_out.write(json.dumps(json_obj) + '\n')

  0%|          | 0/20 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/usuario/.local/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
  5%|▌         | 1/20 [00:35<11:11, 35.34s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'prompt': 'Write a 300+ word summary of the wikipedia page "https://en.wikipedia.org/wiki/Raymond_III,_Count_of_Tripoli". Do not use any commas and highlight at least 3 sections that has titles in markdown format, for example *highlighted section part 1*, *highlighted section part 2*, *highlighted section part 3*.', 'response': ' Question: Write a 300+ word summary of the wikipedia page "https://en.wikipedia.org/wiki/Raymond_III,_Count_of_Tripoli". Do not use any commas and highlight at least 3 sections that has titles in markdown format, for example *highlighted section part 1*, *highlighted section part 2*, *highlighted section part 3*.\n\nAnswer:\n\nRaymond III, Count of Tripoli (1134–1187) was a Crusader leader and the Count of Tripoli from 1152 to 1187. He was the son of Raymond II, Count of Tripoli, and Constance of Antioch.\n\nRaymond III was born in 1134 and succeeded his father as Count of Tripoli in 1152. He was a skilled military leader and played a key role in the Crusades

 10%|█         | 2/20 [01:04<09:35, 31.96s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'prompt': 'I am planning a trip to Japan, and I would like thee to write an itinerary for my journey in a Shakespearean style. You are not allowed to use any commas in your response.', 'response': ' What a fine idea! I shall do my best to craft a most excellent itinerary for your journey to Japan in the style of William Shakespeare.\n\nTo begin, I recommend that you start your journey in the bustling city of Tokyo, where you can marvel at the neon lights and towering skyscrapers. From there, you can take a train to the ancient capital of Kyoto, where you can explore the many temples and shrines that dot the city.\n\nNext, you can head to the coastal city of Hiroshima, where you can visit the Peace Memorial Park and learn about the tragic history of the atomic bombing. After that, you can take a ferry to the island of Miyajima, where you can see the famous floating torii gate and hike up to the top of Mount Misen for a breathtaking view of the surrounding islands.\n\nFrom there, you ca

 15%|█▌        | 3/20 [01:42<09:48, 34.62s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

{'prompt': 'Write two jokes about rockets. Do not contain commas in your response. Separate the two jokes with 6 asterisk symbols: ******.', 'response': " What do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't work? A paperweight.\n\n******\n\nWhat do you call a rocket that doesn't wor

 95%|█████████▌| 19/20 [12:07<00:40, 40.38s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'prompt': 'Are hamburgers sandwiches? Please respond using only the Kannada language, no other language is allowed.', 'response': ' ಹಂಬರ್ಗರ್ ಸಂಪುರ್ಣ ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರು ಹೊಂದಿದೆ. ಅವರ'}


100%|██████████| 20/20 [12:46<00:00, 38.35s/it]

{'prompt': "make a tweet for playboy's twitter account without using capital letters. Include at least 4 hashtags, starting with '#'", 'response': " What's up, ladies? We've got some hot new content for you this week. Check out our latest issue for some steamy photos and articles that will make your heart race. Don't forget to follow us on Instagram for more exclusive content. #playboy #sex #hot #sexy #photos #models #magazine #content #follow #instagram #exclusive #photos #models #magazine #content #follow #instagram #exclusive #photos #models #magazine #content #follow #instagram #exclusive #photos #models #magazine #content #follow #instagram #exclusive #photos #models #magazine #content #follow #instagram #exclusive #photos #models #magazine #content #follow #instagram #exclusive #photos #models #magazine #content #follow #instagram #exclusive #photos #models #magazine #content #follow #instagram #exclusive #photos #models #magazine #content #follow #instagram #exclusive #photos #m